In [160]:
import numpy as np
import pandas as pd
import json
import re
import matplotlib.pyplot as plt

In [161]:
#显示所有列 
pd.set_option('display.max_columns', None) 
#显示所有行 
pd.set_option('display.max_rows', None) 
#设置value的显示长度为100，默认为50 
pd.set_option('max_colwidth',100) 

In [162]:
bus_data = pd.read_csv('/home/lihuan/dgg/bss-recommended-platform-dev/dgg_recommend_push/data/business_df.csv')
user_data = pd.read_csv('/home/lihuan/dgg/bss-recommended-platform-dev/dgg_recommend_push/data/employee_df.csv')
total_data = pd.read_csv('/home/lihuan/dgg/bss-recommended-platform-dev/dgg_recommend_push/data/total_data.csv')

/home/lihuan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,10,12,13,14,15,16,19,21,22,23,24,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [163]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319858 entries, 0 to 319857
Data columns (total 77 columns):
call_duration                 289997 non-null float64
customer_id                   319858 non-null int64
follow_days                   225305 non-null float64
login_name                    319858 non-null int64
product_amount                16037 non-null float64
salary_score                  303821 non-null float64
avgAddWechatFriendCount       319525 non-null float64
avgCallCount                  319854 non-null float64
avgIntentCustomerCount        269739 non-null float64
avgRemarkWords                319797 non-null float64
avgSignBillDays               191921 non-null float64
avgToDoorCount                263787 non-null float64
avgWorkHour                   319858 non-null float64
behaviorScore                 319858 non-null float64
loanRate                      229507 non-null float64
performanceRank               269807 non-null float64
signBillCount                 

## 生成训练label，去掉无用字段

In [164]:
total_data['label'] = total_data['product_amount'].apply(lambda x:1 if x>=0 else 0)
total_data.drop(['call_duration','customer_id','follow_days',
                 'login_name','product_amount','salary_score'],axis =1,inplace=True)

## 提取商务工号，商机id，作为查询索引，同时在数据中删除该列

In [165]:
user_list = list(user_data['login_name'])
bus_list = list(bus_data['customer_id'])
user_data.drop('login_name',axis = 1, inplace = True)
bus_data.drop('customer_id',axis=1,inplace=True)

## 统计正负样本数量，对负样本进行采样

In [166]:
total_data_po = total_data.loc[total_data['label']==1]
total_data_ne = total_data.loc[total_data['label']==0]

In [167]:
#控制负样本与正样本的比例为10:1
samples = total_data_po.shape[0]*9
total_data_ne_sample = total_data_ne.sample(samples)

In [168]:
total_data_sample = pd.concat((total_data_ne_sample,total_data_po),axis=0)
total_data_sample = total_data_sample.sample(frac=1)

In [169]:
total_data_sample.head(10)

,avgAddWechatFriendCount,avgCallCount,avgIntentCustomerCount,avgRemarkWords,avgSignBillDays,avgToDoorCount,avgWorkHour,behaviorScore,loanRate,performanceRank,signBillCount,signBillRate,toDoorRate,age,locked,post,rank,sex,work_age,average_performance_amount,average_profit_amount,average_internal_cost,average_external_cost,average_payment_amount,average_business_amount,average_cost_price,average_CALL_0940_1040,average_CALL_1100_1200,average_CALL_1400_1500,average_CALL_1520_1620,average_achievement,average_profit,average_internal_cost_orf,average_external_cost_orf,average_assist_achievement,average_assist_profit,orderAmount,meetingCount,orderCount,tccs,flagContent,flagRemark,mon,remarkType,times,addTypeCode,allotNumber,businessLocation,businessOperate,businessStage,businessStatus,extensionCode,followTimes,invalidNumber,isCost,isMind,lastLoseType,opportunitytypeCode,originCode,originType,placeCode,referral,uploadType,vip,wayCode,widelyType,coverCharge,refundOrderSign,signOderNum,callTimes,avgCallDuration,label
109203,0.000,33.0,NaN,40.97,0.54,0.60,8.88,106.0,0.5000,0.356522,20.0,2.0000,0.09,24.0,0,商务顾问,P,1,2.0,1598.000000,1311.928571,328.928571,75.714286,1816.571429,6000.000000,55.000000,3.500000,3.833333,3.500000,3.866667,290.142857,235.321429,54.821429,12.619048,0.0,0.0,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_4,0.0,emp,BUS_3,NaN,BUS_STA_2_3,NaN,11.0,0.0,CBJSFS_NO,0.0,org,BUS_OPP_4,BUS_LYQD_707,ORIGIN_TYPE_10,BUS_SOR_PLACE_CQ,0.0,2.0,0.0,BUS_WAY_CODE_5,3.0,NaN,1.0,1.0,7.0,137.29,0
228716,0.000,77.0,1.89,47.17,NaN,1.53,9.97,96.0,NaN,0.008696,24.0,5.0000,0.06,23.0,0,商务顾问,P,0,2.0,4463.000000,4123.000000,425.000000,0.000000,5542.500000,0.000000,0.000000,4.720000,4.680000,4.040000,3.760000,316.714286,292.428571,24.285714,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,22.00,0
187435,1.000,79.0,0.10,37.58,NaN,NaN,9.34,110.5,NaN,0.573913,NaN,NaN,NaN,22.0,0,商务顾问,P,1,10.0,2675.620000,1979.088333,1393.063333,64.972222,5414.990556,39666.944444,12.500000,4.800000,5.758824,10.741176,8.052941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_4,0.0,hid,BUS_707,NaN,BUS_STA_DELETE_CHECK,NaN,1.0,0.0,CBJSFS_NO,0.0,org,BUS_OPP_4,BUS_LYQD_707,ORIGIN_TYPE_10,BUS_SOR_PLACE_CD,0.0,1.0,0.0,BUS_WAY_CODE_5,3.0,NaN,NaN,NaN,1.0,312.00,0
70709,0.909,62.0,1.18,35.78,11.39,0.32,8.90,107.0,0.7143,0.023188,13.0,1.0000,0.11,21.0,0,商务顾问,P,1,6.0,1244.218421,1032.063158,212.155263,53.684211,1388.428947,3266.789474,48.105263,3.452174,5.295652,4.156522,3.721739,309.006311,247.846602,61.159709,4.854369,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_4,0.0,org,BUS_OUT_702,BUS_STAGE_0,BUS_STA_2_2,NaN,1.0,0.0,NaN,0.0,NaN,BUS_WAY_CODE_6,BUS_LYQD_707,ORIGIN_TYPE_11,BUS_SOR_PLACE_CD,0.0,1.0,0.0,BUS_WAY_CODE_3,NaN,NaN,NaN,NaN,10.0,18.20,0
71622,50.240,67.0,0.76,24.79,NaN,0.44,9.09,116.0,0.1364,0.527536,37.0,4.4000,0.05,24.0,0,商务顾问,P,0,6.0,3740.750000,3392.225000,348.525000,0.000000,3740.750000,6710.000000,65.375000,2.546218,1.588235,1.731092,1.336134,406.830000,361.523000,45.457000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,7.00,0
19668,0.870,34.0,0.04,25.78,2.53,0.09,8.93,112.0,0.6364,0.750725,13.0,2.2000,0.00,24.0,0,商务顾问,P,1,9.0,5466.045455,4108.156818,1396.240909,622.818182,6385.681818,8193.863636,50.863636,3.479290,4.402367,3.988166,3.230769,1450.082474,1096.476289,353.606186,157.391753,NaN,NaN,42200.0,0.0,4.0,NaN,0.000000,0.0,0.0,NaN,NaN,BUS_ADD_TYPE_CODE_5,0.0,emp,BUS_4,NaN,BUS_STA_2_3,NaN,4.0,0.0,CBJSFS_NO,0.0,NaN,BUS_OPP_3,BUS_LYQD_101,ORIGIN_TYPE_6,BUS_SOR_PLACE_CD,0.0,2.0,0.0,BUS_WAY_CODE_2,0.0,29400.0,0.0,NaN,17.0,86.06,1
189522,2.864,53.0,1.77,29.86,10.27,0.23,9.16,110.0,0.2143,0.240580,17.0,4.6667,0.03,24.0,0,商务顾问,P,1,2.0,2142.207500,1684.807500,457.400000,0.000000,2142.207500,2872.5

## 对特征进行归类，分为连续型，类别型和布尔型

In [170]:
continues_ccolumns = ['avgAddWechatFriendCount','avgCallCount','avgIntentCustomerCount','avgRemarkWords',
                     'avgSignBillDays','avgToDoorCount','avgWorkHour','behaviorScore','loanRate','performanceRank',
                     'signBillCount','signBillRate','toDoorRate','age','work_age','average_performance_amount',
                     'average_profit_amount','average_internal_cost','average_external_cost','average_payment_amount',
                     'average_business_amount','average_cost_price','average_CALL_0940_1040','average_CALL_1100_1200',
                     'average_CALL_1400_1500','average_CALL_1520_1620','average_achievement','average_profit',
                     'average_internal_cost_orf','average_external_cost_orf','average_assist_achievement',
                     'average_assist_profit','orderAmount','meetingCount','orderCount','tccs','flagContent',
                     'flagRemark','mon','times','allotNumber','followTimes','invalidNumber','signOderNum',
                     'coverCharge','callTimes','avgCallDuration']

In [171]:
category_columns = ['post','rank','sex','remarkType','addTypeCode','businessLocation',
                   'businessOperate','businessStage','businessStatus','extensionCode',
                    'lastLoseType','opportunitytypeCode','originCode','originType',
                    'placeCode','uploadType','wayCode','widelyType']

In [172]:
bool_columns = ['locked','isCost','isMind','referral','vip','refundOrderSign']

In [173]:
len(continues_ccolumns)

47

## 初步筛选无用特征

In [174]:
# 离散型变量,如果某一取值的数量超过了95%，则舍弃该变量
remove = set()
print('处理类别型变量')
for k in category_columns+bool_columns:
    counts = total_data_sample[k].value_counts()
    total_num = sum(counts)
    for v in counts:
        if v/total_num>=0.95:
            print(k)
            print(counts)
            remove.add(k)
# 连续型变量,如果其下四分位数=中位数=上四分位数，则舍弃该变量
print('处理连续型变量')
for k in continues_ccolumns:
    describe = total_data_sample[k].describe()
    if describe[4]==describe[6]:
        remove.add(k)
        print(k,describe[4],describe[6])
# 去掉缺失率大于80%的变量
print('处理缺失率较大的变量')
for k in total_data_sample.keys():
    miss = total_data[k].isna().sum()/total_data.shape[0]
    if miss>0.8:
        remove.add(k)
        print(k,':',miss)
# 去掉商机地区信息
remove.add('placeCode')

处理类别型变量
post
商务顾问    157573
电销专员      2797
Name: post, dtype: int64
locked
0    160370
Name: locked, dtype: int64
isMind
0.0    104105
1.0         9
Name: isMind, dtype: int64
referral
0.0    103470
1.0       639
Name: referral, dtype: int64
vip
0.0    104167
1.0       984
Name: vip, dtype: int64
处理连续型变量
average_assist_achievement 0.0 0.0
average_assist_profit 0.0 0.0
orderCount 1.0 1.0
tccs 1.0 1.0
flagContent 0.0 0.0
flagRemark 1.0 1.0
invalidNumber 0.0 0.0
signOderNum 1.0 1.0
处理缺失率较大的变量
orderAmount : 0.919458009491712
meetingCount : 0.919458009491712
orderCount : 0.919458009491712
tccs : 0.9984837021428259
mon : 0.9713560392424139
remarkType : 0.980038016869986
times : 0.980038016869986
businessStage : 0.9054174039730131
extensionCode : 0.9922621913474104
coverCharge : 0.9662256376266969
refundOrderSign : 0.8206641697253156
signOderNum : 0.8703987394406267


In [175]:
user_remove = []
bus_remove = []
for k in remove:
    if k in user_data.keys():
        user_remove.append(k)
    else:
        bus_remove.append(k)
total_data_sample.drop(list(remove),axis=1,inplace=True)
user_data.drop(user_remove,axis=1,inplace=True)
bus_data.drop(bus_remove,axis=1,inplace=True)

In [176]:
remove

{'average_assist_achievement',
 'average_assist_profit',
 'businessStage',
 'coverCharge',
 'extensionCode',
 'flagContent',
 'flagRemark',
 'invalidNumber',
 'isMind',
 'locked',
 'meetingCount',
 'mon',
 'orderAmount',
 'orderCount',
 'placeCode',
 'post',
 'referral',
 'refundOrderSign',
 'remarkType',
 'signOderNum',
 'tccs',
 'times',
 'vip'}

检查历史数据，商务数据，商机数据标签正确性

In [177]:
list(total_data_sample.keys()[:-1]) == list(user_data.keys())+list(bus_data.keys())

True

In [178]:
print(user_data.shape)
print(bus_data.shape)

(469, 32)
(1120737, 16)


## 简单处理缺失值

In [190]:
import sklearn
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [180]:
#连续变量用0填充,类别型变量全部用‘None’填充
def fillna(df):
    for k in df.keys():
        if k in continues_ccolumns:
            df[k] = df[k].fillna(0)
        else:
            df[k] = df[k].fillna('None')

In [181]:
fillna(total_data_sample)
fillna(user_data)
fillna(bus_data)

## 数据预处理，连续变量采用minmax，类别型变量采用onehot

In [182]:
# 对连续变量进行区间缩放并存放中间值,对类别型变量进行独热编码并存放中间值
continues_dict = {}
category_dict = {}
for k in total_data_sample.keys():
    if k in continues_ccolumns:
        button = min(total_data_sample[k])
        top = max(total_data_sample[k])
        total_data_sample[k] = total_data_sample[k].apply(lambda x:(x-button)/(top-button))
        continues_dict[k] = (button,top)
    elif k in category_columns or k in bool_columns:
        category_dict[k] = list(total_data_sample[k].unique())
        for v in category_dict[k]:
            total_data_sample[k+'_'+str(v)] = total_data_sample[k].apply(lambda x:1 if x ==v else 0)
        total_data_sample.drop(k,axis=1,inplace=True)

In [183]:
model = LogisticRegression(C=0.1,penalty='l1',multi_class='ovr',tol=1e-5,n_jobs=4)

In [184]:
label = total_data_sample['label']
total_data_sample.drop('label',axis = 1,inplace=True)
lenth = total_data_sample.shape[0]

In [185]:
train_data = total_data_sample[:int(0.8*lenth)]
train_label = label[:int(0.8*lenth)]
test_data = total_data_sample[int(0.8*lenth):]
test_label = label[int(0.8*lenth):]

In [192]:
model.fit(train_data,train_label)

/home/lihuan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lihuan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=4,
          penalty='l1', random_state=None, solver='warn', tol=1e-05,
          verbose=0, warm_start=False)

In [193]:
y_pre = model.predict(test_data)

In [202]:
metrics.recall_score(test_label,y_pre)

0.674812030075188

In [200]:
y_train = model.predict(train_data)

In [203]:
metrics.recall_score(train_label,y_train)

0.6692098092643052

In [ ]:
test_data_bus = 

分别提取正负样本，并对副样本进行抽样

In [48]:
total_data_po = total_data.loc[total_data['label']==1]
total_data_ne = total_data.loc[total_data['label']==0]

In [52]:
total_data_po.shape[0]/total_data_ne.shape[0]

0.05278436974402691

In [ ]:
sc = schdule_load()
sc.main(run, 5)

2019-07-09 16:40:03
2019-07-09 16:40:03	==========开始读取通话记录==============
2019-07-09 16:40:03	初始化connection
2019-07-09 16:40:04	初始化connection
2019-07-09 16:40:04	查询到7条数据:
2019-07-09 16:40:22	(962598, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962598 entries, 0 to 962597
Data columns (total 3 columns):
login_name       962598 non-null object
customer_id      962598 non-null int64
call_duration    962598 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 22.0+ MB
2019-07-09 16:40:22	None
2019-07-09 16:40:22	=========开始读取跟进记录===========
2019-07-09 16:40:22	初始化connection
2019-07-09 16:40:36	(351877, 6)
2019-07-09 16:40:36	(174491, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 174491 entries, 0 to 351655
Data columns (total 6 columns):
customer_id    174491 non-null int64
business_id    174491 non-null int64
login_name     174491 non-null object
content        174491 non-null object
followStart    174491 non-null datetime64[ns]
followEnd      174491 non-n

/Users/luocheng/Documents/dgg/bss-recommended-platform/loans_drop/data_cleaning/load_data.py:321: FutureWarning: Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version
  loader.pull_total_data()


2019-07-09 16:40:36	(468190, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 468190 entries, 0 to 468189
Data columns (total 3 columns):
login_name       468190 non-null object
customer_id      468190 non-null int64
call_duration    468190 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.3+ MB
2019-07-09 16:40:36	None
2019-07-09 16:40:36	  login_name  customer_id  call_duration
0     100008       138905           17.0
1     100008      2018095          161.0
2     100008      2349605            8.0
3     100008      2443785          211.0
4     100008      2783825           15.5
2019-07-09 16:40:36	============跟进记录进行groupby============
2019-07-09 16:40:54	(154891, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 154891 entries, 0 to 154890
Data columns (total 6 columns):
login_name     154891 non-null object
business_id    154891 non-null int64
customer_id    154891 non-null int64
content        154891 non-null object
follow_days    154891 non-null flo

/Users/luocheng/Documents/dgg/bss-recommended-platform/loans_drop/data_cleaning/load_data.py:254: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  merge_df = pd.concat([order_df, merge_df], axis=0)


2019-07-09 16:42:44	============实时读取客户基本信息=============
2019-07-09 16:42:44	开始合并商机数据
2019-07-09 16:42:44	read_dgg_customer总共有0条商机数据
2019-07-09 16:42:44	(0, 0)
2019-07-09 16:42:44	Empty DataFrame
Columns: []
Index: []
2019-07-09 16:42:44	Index([], dtype='object')
2019-07-09 16:42:44	error	business_MongoDB.py	193	get_customer_feature	['        print_to_log(e, level=4)\n']	0	"None of [Index(['customer_id', 'meetingCount', 'orderAmount', 'orderCount'], dtype='object')] are in the [columns]"
2019-07-09 16:42:44
2019-07-09 16:42:45	read_customer_tccs_entity总共有0条商机数据
2019-07-09 16:42:45	(0, 0)
2019-07-09 16:42:45	Empty DataFrame
Columns: []
Index: []
2019-07-09 16:42:45	Index([], dtype='object')
2019-07-09 16:42:45	error	business_MongoDB.py	209	get_customer_feature	['        print_to_log(e, level=4)\n']	0	'customer_id'
2019-07-09 16:42:45
2019-07-09 16:43:30	read_last_ninety_customer_data总共有2324159条商机数据
2019-07-09 16:43:33	(2324159, 4)
2019-07-09 16:43:33	   createTime  customerId  flagConten